# 08 - Transformação de Numéricas para Categóricas com Decision Trees

## Importação

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRFRegressor
%matplotlib inline
from sklearn.metrics import median_absolute_error

## Carga dos Dados

In [ ]:
train = pd.read_csv("data-processed/train.csv")
val = pd.read_csv("data-processed/val.csv")

train['DATE_TIME'] = pd.to_datetime(train['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')
val['DATE_TIME'] = pd.to_datetime(val['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
train2 = train.copy()
train2['WEEKDAY'] = train2['DATE_TIME'].dt.weekday
train2['HOUR'] = train2['DATE_TIME'].dt.hour
train2['MINUTE'] = train2['DATE_TIME'].dt.minute

val2 = val.copy()
val2['WEEKDAY'] = val2['DATE_TIME'].dt.weekday
val2['HOUR'] = val2['DATE_TIME'].dt.hour
val2['MINUTE'] = val2['DATE_TIME'].dt.minute

## Implementando

In [ ]:
mdl = XGBRFRegressor(random_state=0, n_estimators=100, tree_method='gpu_hist', gpu_id=0, subsample=0.6, colsample_bynode=0.4)

Xtr, ytr = train2[features], train2['Y4WIN']

mdl.fit(Xtr,ytr)

In [ ]:
?mdl.apply

In [ ]:
leaves = mdl.apply(Xtr)
print(leaves.shape)
leaves

In [ ]:
from  sklearn.preprocessing import OneHotEncoder
leaves_ohe = OneHotEncoder()
leaves_train_ohe = leaves_ohe.fit_transform(leaves)


leaves_val = mdl.apply(val2[features])
leaves_val_ohe = leaves_ohe.transform(leaves_val)

In [ ]:
from sklearn.linear_model import Ridge, Lasso

mdl2 = Lasso(18)
mdl2.fit(leaves_train_ohe, train2['Y4WIN'])


p = mdl2.predict(leaves_val_ohe)
median_absolute_error(val2['Y4'], p)

# Fim